In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("/home/shigilsasi/code/Guvi_Projects/Axis_Bank_Intelligent_Product_&_Offer_Recommendation_System_using_Machine_Learning/output/axis_transactions.csv")
df.head()

,account_id,date,narration,channel,merchant,category,debit,credit,balance,source_file
0,0001_Thiyagarajan_Subbu_Statement,2022-12-01,UPI/973571737/LANDLORD RENT/landlord@okhdfc,UPI,LANDLORD RENT,Rent,15139.0,0.0,203044.0,0001_Thiyagarajan_Subbu_Statement.pdf
1,0001_Thiyagarajan_Subbu_Statement,2022-12-01,POS/988002/AMAZON INDIA/CARD **9369,POS,AMAZON INDIA,Shopping,3603.0,0.0,199441.0,0001_Thiyagarajan_Subbu_Statement.pdf
2,0001_Thiyagarajan_Subbu_Statement,2022-12-01,UPI/957858928/RAPIDO/rapido@ybl,UPI,RAPIDO,Transport,135.0,0.0,199306.0,0001_Thiyagarajan_Subbu_Statement.pdf
3,0001_Thiyagarajan_Subbu_Statement,2022-12-03,POS/214232/FLIPKART INDIA/CARD **9768,POS,FLIPKART INDIA,Shopping,1998.0,0.0,197308.0,0001_Thiyagarajan_Subbu_Statement.pdf
4,0001_Thiyagarajan_Subbu_Statement,2022-12-04,UPI/233699231/EATSURE/eatsure@hdfcbank,UPI,EATSURE,Other,374.0,0.0,196934.0,0001_Thiyagarajan_Subbu_Statement.pdf


In [4]:
df.columns

Index(['account_id', 'date', 'narration', 'channel', 'merchant', 'category',
       'debit', 'credit', 'balance', 'source_file'],
      dtype='object')

In [5]:
df.dtypes

account_id      object
date            object
narration       object
channel         object
merchant        object
category        object
debit          float64
credit         float64
balance        float64
source_file     object
dtype: object

In [6]:
df['merchant'].value_counts()

merchant
ZOMATO             85215
SWIGGY             84729
EATSURE            84617
OLA                56315
RAPIDO             56232
                   ...  
ACNO 9720363461        1
SIP909387              1
VL66096                1
ACNO 9087325699        1
SIP779180              1
Name: count, Length: 135880, dtype: int64

In [7]:
df['category'].unique()

array(['Rent', 'Shopping', 'Transport', 'Other', 'Food & Dining',
       'Entertainment', 'Investment', 'Utilities', 'Bank Charges',
       'ATM Cash Withdrawal', 'Salary', 'Interest Credit', 'Transfer',
       'Insurance', 'Fuel'], dtype=object)

In [8]:
df_other = df[df['category'] == 'Other']
df_other.head()

,account_id,date,narration,channel,merchant,category,debit,credit,balance,source_file
4,0001_Thiyagarajan_Subbu_Statement,2022-12-04,UPI/233699231/EATSURE/eatsure@hdfcbank,UPI,EATSURE,Other,374.0,0.0,196934.00,0001_Thiyagarajan_Subbu_Statement.pdf
5,0001_Thiyagarajan_Subbu_Statement,2022-12-04,UPI/503872733/SWIGGY/swiggy@hdfcbank,UPI,SWIGGY,Other,341.0,0.0,196593.00,0001_Thiyagarajan_Subbu_Statement.pdf
12,0001_Thiyagarajan_Subbu_Statement,2022-12-11,UPI/677898986/ZOMATO/zomato@hdfcbank,UPI,ZOMATO,Other,417.0,0.0,177592.45,0001_Thiyagarajan_Subbu_Statement.pdf
15,0001_Thiyagarajan_Subbu_Statement,2022-12-15,UPI/266011009/SPAR HYPERMARKET CBE/sparhyperma...,UPI,SPAR HYPERMARKET CBE,Other,3334.0,0.0,167370.45,0001_Thiyagarajan_Subbu_Statement.pdf
16,0001_Thiyagarajan_Subbu_Statement,2022-12-15,UPI/635472868/BOOKMYSHOW/bms@axisbank,UPI,BOOKMYSHOW,Other,573.0,0.0,166797.45,0001_Thiyagarajan_Subbu_Statement.pdf


In [9]:
df_other.sample(100)

,account_id,date,narration,channel,merchant,category,debit,credit,balance,source_file
1072947,0778_Durai_Murugan_Statement,2025-07-25,UPI/218579015/EATSURE/eatsure@hdfcbank,UPI,EATSURE,Other,318.0,0.0,-43788.73,0778_Durai_Murugan_Statement.pdf
1233786,0890_Wasim_Balaji_Statement,2024-08-27,UPI/305974418/ZOMATO/zomato@hdfcbank,UPI,ZOMATO,Other,428.0,0.0,1493278.36,0890_Wasim_Balaji_Statement.pdf
913357,0659_Yamuna_Pandian_Statement,2023-08-13,POS/359850/SHREE ANANDHAAS/CARD **7876,POS,SHREE ANANDHAAS,Other,415.0,0.0,1206351.83,0659_Yamuna_Pandian_Statement.pdf
735104,0526_Ganesh_Vijay_Statement,2025-11-05,UPI/591619397/NILGIRIS CBE/nilgiriscbe@upi,UPI,NILGIRIS CBE,Other,1960.0,0.0,334575.13,0526_Ganesh_Vijay_Statement.pdf
1284472,0925_Hari_Babu_Statement,2023-08-23,UPI/577248137/SWIGGY/swiggy@hdfcbank,UPI,SWIGGY,Other,714.0,0.0,620177.32,0925_Hari_Babu_Statement.pdf
...,...,...,...,...,...,...,...,...,...,...
917172,0662_Manjula_Palani_Statement,2024-12-10,EMI/AXISCARLOAN/VL55193/VEHICLE LOAN,OTHER,VL55193,Other,5992.0,0.0,-146985.31,0662_Manjula_Palani_Statement.pdf
700746,0502_Saranya_Durai_Statement,2023-12-10,EMI/AXISHOMELOAN/LN19252/HOME LOAN,OTHER,LN19252,Other,24130.0,0.0,-24698.34,0502_Saranya_Durai_Statement.pdf
1313143,0947_Shankar_Balaji_Statement,2024-03-18,UPI/589559974/EATSURE/eatsure@hdfcbank,UPI,EATSURE,Other,520.0,0.0,-785618.20,0947_Shankar_Balaji_Statement.pdf
286661,0205_Lalitha_Kumar_Statement,2025-10-07,UPI/987085996/MEDPLUS CBE/medpluscbe@ybl,UPI,MEDPLUS CBE,Other,383.0,0.0,3643132.19,0205_Lalitha_Kumar_Statement.pdf


In [10]:
df_other['merchant'].value_counts()

merchant
ZOMATO                85215
SWIGGY                84729
EATSURE               84617
ACT FIBERNET          23412
RELIANCE FRESH CBE    17077
                      ...  
LN68991                   1
LN89667                   1
LN29412                   1
LN76328                   1
LN11866                   1
Name: count, Length: 19215, dtype: int64

In [11]:
FOOD_MERCHANTS = [
    "zomato", "swiggy", "eatsure", "restaurant", "hotel",
    "biryani", "mess", "annandhaas", "annapoorna"
]

GROCERY_MERCHANTS = [
    "reliance fresh", "spar", "pazhamudir", "nilgiris",
    "supermarket", "hypermarket"
]

TRANSPORT_MERCHANTS = ["ola", "uber", "rapido"]


In [12]:
def recategorize_other(row):
    text = f"{row['merchant']} {row['narration']}".lower()

    # Food & Dining
    if any(m in text for m in FOOD_MERCHANTS):
        return "Food & Dining"

    # Groceries
    if any(m in text for m in GROCERY_MERCHANTS):
        return "Groceries"

    # Transport
    if any(m in text for m in TRANSPORT_MERCHANTS):
        return "Transport"

    # Utilities
    if any(x in text for x in ["electricity", "water", "gas", "tangedco", "billpay"]):
        return "Utilities"

    # Rent
    if "rent" in text or "landlord" in text:
        return "Rent"

    # Investment
    if "sip" in text or "mutual fund" in text:
        return "Investment"

    # Bank Charges
    if "charge" in text or "sms alert" in text:
        return "Bank Charges"

    # Interest
    if "interest" in text or "int/credit" in text:
        return "Interest Credit"

    # Transfers
    if any(x in text for x in ["upi", "neft", "imps", "rtgs"]):
        return "Transfer"

    return "Unknown Merchant"


In [13]:
mask = df['category'] == 'Other'
df.loc[mask, 'category'] = df[mask].apply(recategorize_other, axis=1)
df['category'].value_counts()

category
Food & Dining          290831
Shopping               189742
Transport              168756
ATM Cash Withdrawal    135264
Unknown Merchant       106302
Groceries              101670
Transfer                99452
Utilities               70852
Fuel                    58628
Bank Charges            45408
Investment              34584
Salary                  33807
Rent                    17383
Entertainment           16852
Interest Credit         11245
Insurance                3259
Name: count, dtype: int64

In [14]:
import re

def normalize_merchant(row):
    text = row['narration'].lower()
    merchant = row['merchant']

    # Electricity
    if "tangedco" in text:
        return "TANGEDCO"

    # SIP / Mutual Fund
    if text.startswith("sip") or "mutual fund" in text:
        return "AXIS MUTUAL FUND"

    # ATM
    if row['category'] == "ATM Cash Withdrawal":
        return "ATM Withdrawal"

    # Bank Charges
    if row['category'] == "Bank Charges":
        return "Axis Bank"

    # Interest
    if row['category'] == "Interest Credit":
        return "Axis Bank"

    # Transfers
    if row['category'] == "Transfer":
        return "Personal Transfer"

    # Remove ACNO / numeric junk
    if re.search(r'acno|\d{6,}', merchant.lower()):
        return "Utility / Reference"

    return merchant.title()


In [15]:
df['clean_merchant'] = df.apply(normalize_merchant, axis=1)

In [16]:
df['merchant'] = df['clean_merchant']
df.drop(columns=['clean_merchant'], inplace=True)

In [17]:
def normalize_system_transactions(row):
    text = row['narration'].lower()

    # Credit Card Bill Payment
    if "axis credit card" in text:
        row['category'] = "Credit Card Payment"
        row['merchant'] = "Axis Credit Card"
        return row

    # Home Loan EMI
    if text.startswith("emi") and "home loan" in text:
        row['category'] = "Loan EMI"
        row['merchant'] = "Axis Home Loan"
        return row

    # Car Loan EMI
    if text.startswith("emi") and "car loan" in text:
        row['category'] = "Loan EMI"
        row['merchant'] = "Axis Car Loan"
        return row

    return row


In [18]:
df = df.apply(normalize_system_transactions, axis=1)

In [19]:
def normalize_emi_transactions(row):
    text = str(row['narration']).lower()
    merchant = str(row['merchant']).upper()

    if text.startswith("emi"):
        row['category'] = "Loan EMI"

        if merchant.startswith("PL"):
            row['merchant'] = "Axis Personal Loan"

        elif merchant.startswith("VL"):
            row['merchant'] = "Axis Vehicle Loan"

        elif merchant.startswith("LN"):
            row['merchant'] = "Axis Home Loan"

        else:
            row['merchant'] = "Axis Loan"

    return row

In [20]:
df = df.apply(normalize_emi_transactions, axis=1)

In [21]:
import re

def normalize_salary_transactions(row):
    text = str(row['narration']).upper()

    if row['category'] == "Salary" and "SALARY" in text:
        # Extract employer name between NEFT-xxxx- and -SALARY
        match = re.search(r'NEFT-\d+-(.*?)-SALARY', text)
        if match:
            employer = match.group(1).title()
            row['merchant'] = employer
        else:
            row['merchant'] = "Employer"

    return row


In [22]:
df = df.apply(normalize_salary_transactions, axis=1)

In [23]:
df['merchant'].value_counts()

merchant
ATM Withdrawal         135264
Personal Transfer       99452
Zomato                  85215
Swiggy                  84729
Eatsure                 84617
                        ...  
Tcs India Pvt Ltd         651
Infosys Limited           608
Suguna Foods              597
Coimbatore Textiles       559
Mindtree Ltd              513
Name: count, Length: 86, dtype: int64

In [24]:
df['category'].value_counts()   

category
Food & Dining          290831
Shopping               189742
Transport              168756
ATM Cash Withdrawal    135264
Groceries              101670
Transfer                99452
Unknown Merchant        86181
Fuel                    58628
Utilities               50637
Bank Charges            45408
Investment              34584
Salary                  33807
Credit Card Payment     20215
Loan EMI                20121
Rent                    17383
Entertainment           16852
Interest Credit         11245
Insurance                3259
Name: count, dtype: int64

In [25]:
df.head()

,account_id,date,narration,channel,merchant,category,debit,credit,balance,source_file
0,0001_Thiyagarajan_Subbu_Statement,2022-12-01,UPI/973571737/LANDLORD RENT/landlord@okhdfc,UPI,Landlord Rent,Rent,15139.0,0.0,203044.0,0001_Thiyagarajan_Subbu_Statement.pdf
1,0001_Thiyagarajan_Subbu_Statement,2022-12-01,POS/988002/AMAZON INDIA/CARD **9369,POS,Amazon India,Shopping,3603.0,0.0,199441.0,0001_Thiyagarajan_Subbu_Statement.pdf
2,0001_Thiyagarajan_Subbu_Statement,2022-12-01,UPI/957858928/RAPIDO/rapido@ybl,UPI,Rapido,Transport,135.0,0.0,199306.0,0001_Thiyagarajan_Subbu_Statement.pdf
3,0001_Thiyagarajan_Subbu_Statement,2022-12-03,POS/214232/FLIPKART INDIA/CARD **9768,POS,Flipkart India,Shopping,1998.0,0.0,197308.0,0001_Thiyagarajan_Subbu_Statement.pdf
4,0001_Thiyagarajan_Subbu_Statement,2022-12-04,UPI/233699231/EATSURE/eatsure@hdfcbank,UPI,Eatsure,Food & Dining,374.0,0.0,196934.0,0001_Thiyagarajan_Subbu_Statement.pdf


In [27]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

df_model = df.drop(columns=[
    'source_file',
    'narration',
    'date',
    'balance'
],errors = 'ignore')

In [29]:
user_features = (
    df_model
    .groupby('account_id')
    .agg(
        # Overall Behavior
        total_debit = ('debit', 'sum'),
        total_credit = ('credit', 'sum'),
        total_transactions = ('debit', 'count'),

        # Category-wise spend
        food_spend = ('debit', lambda x: x[df_model.loc[x.index, 'category'] == 'Food & Dining'].sum()),
        shopping_spend = ('debit', lambda x: x[df_model.loc[x.index, 'category'] == 'Shopping'].sum()),
        transport_spend = ('debit', lambda x: x[df_model.loc[x.index, 'category'] == 'Transport'].sum()),
        rent_spend = ('debit', lambda x: x[df_model.loc[x.index, 'category'] == 'Rent'].sum()),
        emi_spend = ('debit', lambda x: x[df_model.loc[x.index, 'category'] == 'Loan EMI'].sum()),
        utility_spend = ('debit', lambda x: x[df_model.loc[x.index, 'category'] == 'Utilities'].sum()),
        
        # Channel Usage
        upi_txn = ('channel', lambda x: (x == 'UPI').sum()),
        pos_txn = ('channel', lambda x: (x == 'POS').sum()),
        neft_txn = ('channel', lambda x: (x == 'NEFT').sum()),

        # merchant affinity
        top_merchant = ('merchant', lambda x: x.value_counts().idxmax())
    )
    .reset_index()
    
)

In [32]:
# Derived Behavorial Ratios

user_features['savings_ratio'] = (
    user_features['total_credit'] /
    (user_features['total_debit'] + 1)
)

user_features['emi_ratio'] = (
    user_features['emi_spend'] /
    (user_features['total_credit'] + 1)
)

user_features['food_ratio'] = (
    user_features['food_spend'] /
    (user_features['total_debit'] + 1)
)

user_features['digital_ratio'] = (
    (user_features['upi_txn'] + user_features['pos_txn']) /
    (user_features['total_transactions'] + 1)
)


In [33]:
print(user_features.shape)
user_features.head()

(1000, 18)


,account_id,total_debit,total_credit,total_transactions,food_spend,shopping_spend,transport_spend,rent_spend,emi_spend,utility_spend,upi_txn,pos_txn,neft_txn,top_merchant,savings_ratio,emi_ratio,food_ratio,digital_ratio
0,0001_Thiyagarajan_Subbu_Statement,4732053.13,4541729.76,1711,219553.0,1017929.0,42917.0,630002.0,0.0,100049.0,870,449,61,ATM Withdrawal,0.959780,0.000000,0.046397,0.770444
1,0002_Aarti_Selvam_Statement,970874.74,961789.01,499,58525.0,263085.0,13628.0,0.0,0.0,28396.0,239,132,23,ATM Withdrawal,0.990641,0.000000,0.060281,0.742000
2,0003_Radha_Krishnan_Statement,1978346.61,1953447.39,664,92397.0,458422.0,18398.0,393988.0,225120.0,43075.0,350,175,22,ATM Withdrawal,0.987414,0.115242,0.046704,0.789474
3,0004_Loganathan_Surya_Statement,2146990.98,3025475.44,960,66430.0,148909.0,36882.0,635993.0,0.0,51404.0,475,219,44,ATM Withdrawal,1.409169,0.000000,0.030941,0.722164
4,0005_Meena_Balaji_Statement,4934117.48,3925762.77,1863,246707.0,1426179.0,47303.0,608564.0,101280.0,111455.0,924,505,69,ATM Withdrawal,0.795636,0.025799,0.050000,0.766631


In [34]:
user_features.shape

(1000, 18)